<a href="https://colab.research.google.com/github/MUKESHRAJMAHENDRAN/Deep-Learning--fastai/blob/main/fastai_tabular_beg.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Working on tabular data 

In [ ]:
!nvidia-smi

In [ ]:
!pip install fastai --upgrade

In [5]:
from fastai.tabular.all import *

In [6]:
path = untar_data(URLs.ADULT_SAMPLE)

In [7]:
path.ls()

(#3) [Path('/root/.fastai/data/adult_sample/export.pkl'),Path('/root/.fastai/data/adult_sample/models'),Path('/root/.fastai/data/adult_sample/adult.csv')]

In [9]:
df = pd.read_csv(path/"adult.csv")
df.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,salary
0,49,Private,101320,Assoc-acdm,12.0,Married-civ-spouse,NaN,Wife,White,Female,0,1902,40,United-States,>=50k
1,44,Private,236746,Masters,14.0,Divorced,Exec-managerial,Not-in-family,White,Male,10520,0,45,United-States,>=50k
2,38,Private,96185,HS-grad,NaN,Divorced,NaN,Unmarried,Black,Female,0,0,32,United-States,<50k
3,38,Self-emp-inc,112847,Prof-school,15.0,Married-civ-spouse,Prof-specialty,Husband,Asian-Pac-Islander,Male,0,0,40,United-States,>=50k
4,42,Self-emp-not-inc,82297,7th-8th,NaN,Married-civ-spouse,Other-service,Wife,Black,Female,0,0,50,United-States,<50k


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32561 entries, 0 to 32560
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   age             32561 non-null  int64  
 1   workclass       32561 non-null  object 
 2   fnlwgt          32561 non-null  int64  
 3   education       32561 non-null  object 
 4   education-num   32074 non-null  float64
 5   marital-status  32561 non-null  object 
 6   occupation      32049 non-null  object 
 7   relationship    32561 non-null  object 
 8   race            32561 non-null  object 
 9   sex             32561 non-null  object 
 10  capital-gain    32561 non-null  int64  
 11  capital-loss    32561 non-null  int64  
 12  hours-per-week  32561 non-null  int64  
 13  native-country  32561 non-null  object 
 14  salary          32561 non-null  object 
dtypes: float64(1), int64(5), object(9)
memory usage: 3.7+ MB


In [11]:
df.isna().sum()

age                 0
workclass           0
fnlwgt              0
education           0
education-num     487
marital-status      0
occupation        512
relationship        0
race                0
sex                 0
capital-gain        0
capital-loss        0
hours-per-week      0
native-country      0
salary              0
dtype: int64

In [18]:
categorical_col = [col for col in df if df[col].dtype == "O"]
continous_col = [col for col in df if df[col].dtype != "O"]

In [26]:
categorical_col.remove("salary")

In [27]:
categorical_col, continous_col

(['workclass',
  'education',
  'marital-status',
  'occupation',
  'relationship',
  'race',
  'sex',
  'native-country'],
 ['age',
  'fnlwgt',
  'education-num',
  'capital-gain',
  'capital-loss',
  'hours-per-week'])

In [28]:
splits = RandomSplitter(valid_pct = 0.2)(range_of(df))

In [29]:
splits

((#26049) [9529,11951,27016,15732,9578,25914,21049,32276,12457,11599...],
 (#6512) [8465,6701,24118,2554,1275,19096,22829,18143,6212,23383...])

In [32]:
to = TabularPandas(df, 
                   procs = [Categorify, FillMissing, Normalize],
                   cat_names = categorical_col,
                   cont_names = continous_col,
                   y_names = "salary",
                   splits = splits
                   )

In [40]:
to.xs.iloc[:3]

,workclass,education,marital-status,occupation,relationship,race,sex,native-country,education-num_na,age,fnlwgt,education-num,capital-gain,capital-loss,hours-per-week
9529,8,12,3,12,1,5,2,40,1,2.376330,0.199330,-0.417679,-0.147037,-0.216957,-1.016139
11951,5,12,5,11,2,5,1,40,1,-1.435648,-0.840233,-0.417679,-0.147037,-0.216957,-0.040407
27016,5,16,3,5,1,5,2,40,1,-1.215726,-0.822807,-0.026097,-0.147037,-0.216957,0.366147


In [39]:
type(to)

fastai.tabular.core.TabularPandas

In [42]:
dls = to.dataloaders(bs =64)

In [45]:
dls.show_batch()

,workclass,education,marital-status,occupation,relationship,race,sex,native-country,education-num_na,age,fnlwgt,education-num,capital-gain,capital-loss,hours-per-week,salary
0,Private,HS-grad,Married-civ-spouse,Craft-repair,Husband,White,Male,United-States,False,35.0,57640.005118,9.0,0.000002,0.000001,35.0,<50k
1,Private,Some-college,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,Mexico,False,28.0,175261.999954,10.0,0.000002,0.000001,40.0,<50k
2,Private,HS-grad,Married-spouse-absent,Handlers-cleaners,Not-in-family,White,Male,Guatemala,False,34.0,499248.997611,9.0,0.000002,0.000001,40.0,<50k
3,Private,HS-grad,Divorced,Craft-repair,Not-in-family,White,Male,United-States,False,37.0,205339.000391,9.0,0.000002,0.000001,49.0,<50k
4,Private,Some-college,Married-civ-spouse,Exec-managerial,Husband,White,Male,United-States,False,34.0,287315.002781,10.0,0.000002,0.000001,55.0,>=50k
5,State-gov,Assoc-acdm,Never-married,Adm-clerical,Own-child,Black,Female,United-States,False,31.0,255830.000497,12.0,0.000002,0.000001,45.0,<50k
6,Private,HS-grad,Divorced,Tech-support,Not-in-family,Black,Female,United-States,False,56.0,135458.000527,9.0,0.000002,0.000001,40.0,<50k
7,Private,HS-grad,Divorced,Tech-support,Unmarried,White,Female,United-States,False,59.0,109014.997211,9.0,0.000002,0.000001,40.0,<50k
8,State-gov,Bachelors,Married-civ-spouse,Prof-specialty,Husband,White,Male,United-States,False,41.0,144927.998583,13.0,0.000002,0.000001,40.0,>=50k
9,Private,HS-grad,Never-married,Sales,Not-in-family,White,Female,United-States,False,20.0,105312.002597,9.0,0.000002,0.000001,35.0,<50k


In [46]:
learn = tabular_learner(dls, metrics = accuracy)

In [52]:
learn.fit_one_cycle(4)

epoch,train_loss,valid_loss,accuracy,time
0,0.344936,0.342879,0.837377,00:07
1,0.322397,0.335309,0.837838,00:05
2,0.315796,0.325426,0.845977,00:05
3,0.318914,0.317539,0.849969,00:05


In [54]:
learn.show_results()

,workclass,education,marital-status,occupation,relationship,race,sex,native-country,education-num_na,age,fnlwgt,education-num,capital-gain,capital-loss,hours-per-week,salary,salary_pred
0,5.0,12.0,1.0,15.0,2.0,5.0,2.0,40.0,1.0,0.690263,-0.435772,-0.417679,-0.147037,-0.216957,-0.040407,0.0,0.0
1,3.0,8.0,3.0,11.0,6.0,1.0,1.0,40.0,1.0,0.250419,-1.518703,0.757069,-0.147037,-0.216957,-0.040407,0.0,1.0
2,5.0,16.0,5.0,13.0,4.0,5.0,1.0,40.0,1.0,-1.215726,-0.595931,-0.026097,-0.147037,-0.216957,-1.666626,0.0,0.0
3,5.0,2.0,7.0,13.0,4.0,3.0,1.0,40.0,1.0,-1.215726,1.155715,-1.200845,-0.147037,-0.216957,-0.446962,0.0,0.0
4,5.0,10.0,3.0,2.0,1.0,5.0,2.0,40.0,1.0,0.030498,0.912217,1.148652,-0.147037,-0.216957,-0.040407,1.0,1.0
5,2.0,3.0,3.0,3.0,1.0,5.0,2.0,40.0,1.0,-0.336039,-0.159768,-0.809262,-0.147037,-0.216957,-0.040407,0.0,0.0
6,5.0,12.0,3.0,2.0,6.0,5.0,1.0,40.0,1.0,-1.362341,-1.558239,-0.417679,-0.147037,-0.216957,-1.666626,0.0,0.0
7,5.0,13.0,3.0,5.0,1.0,5.0,2.0,40.0,1.0,0.470341,0.258544,1.540235,-0.147037,-0.216957,-0.040407,1.0,1.0
8,5.0,16.0,5.0,7.0,4.0,5.0,2.0,40.0,1.0,-1.362341,2.142828,-0.026097,-0.147037,-0.216957,-0.040407,0.0,0.0


In [55]:
#predict on one row
row, clas, probs = learn.predict(df.iloc[0])

In [56]:
row.show()

,workclass,education,marital-status,occupation,relationship,race,sex,native-country,education-num_na,age,fnlwgt,education-num,capital-gain,capital-loss,hours-per-week,salary
0,Private,Assoc-acdm,Married-civ-spouse,#na#,Wife,White,Female,United-States,False,49.0,101320.001638,12.0,0.000002,1901.999981,40.0,>=50k


In [60]:
row1, clas, probs = learn.predict(df.iloc[4])

In [61]:
row1.show()

,workclass,education,marital-status,occupation,relationship,race,sex,native-country,education-num_na,age,fnlwgt,education-num,capital-gain,capital-loss,hours-per-week,salary
0,Self-emp-not-inc,7th-8th,Married-civ-spouse,Other-service,Wife,Black,Female,United-States,True,42.0,82296.999366,10.0,0.000002,0.000001,50.0,<50k


In [62]:
clas, probs

(tensor(0), tensor([0.7722, 0.2278]))

In [63]:
test_df = df.copy()
test_df.drop(['salary'], axis=1, inplace=True)
dl = learn.dls.test_dl(test_df)

In [64]:
learn.get_preds(dl=dl)

(tensor([[0.1531, 0.8469],
         [0.2313, 0.7687],
         [0.9889, 0.0111],
         ...,
         [0.6030, 0.3970],
         [0.7360, 0.2640],
         [0.7714, 0.2286]]), None)

In [65]:
to.xs[:3]

,workclass,education,marital-status,occupation,relationship,race,sex,native-country,education-num_na,age,fnlwgt,education-num,capital-gain,capital-loss,hours-per-week
9529,8,12,3,12,1,5,2,40,1,2.376330,0.199330,-0.417679,-0.147037,-0.216957,-1.016139
11951,5,12,5,11,2,5,1,40,1,-1.435648,-0.840233,-0.417679,-0.147037,-0.216957,-0.040407
27016,5,16,3,5,1,5,2,40,1,-1.215726,-0.822807,-0.026097,-0.147037,-0.216957,0.366147


In [66]:
X_train, y_train = to.train.xs, to.train.ys.values.ravel()
X_test, y_test = to.valid.xs, to.valid.ys.values.ravel()

In [68]:
from sklearn.ensemble import RandomForestClassifier

In [69]:
model = RandomForestClassifier()

In [70]:
model.fit(X_train, y_train)

RandomForestClassifier()

In [71]:
model.score(X_test, y_test)

0.8556511056511057